# SEIR: Saturation Kinetics Dynamics

**Created by:** Samuel Ropert  
**Creation date:** 04/08/2020  
**Institution:** Computational Biology Lab - Fundación Ciencia y Vida, Chile  

## SEIR
This jupyter notebook explores the saturation kinetics dynamics.


In [1]:
import sys
from pathlib import Path
sys.path.insert(1, '../src/')
sys.path.insert(1, 'src/')
from SEIRmodel import SEIRmodel
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use("TkAgg")
%matplotlib tk
from joblib import Parallel, delayed
import multiprocessing

ModuleNotFoundError: No module named 'SEIRmodel'

## Epidemiological Parameters

In [2]:
beta = 0.15 # Contagion rate
mu = 0 # E0/I0 initial rate
ScaleFactor = 1 # Scale Factor: Number of real infected over reported 
SeroPrevFactor = 1 # Sero Prevalence Factor: Adjust the proportion of the population that enters the virus dynamics
expinfection = 1 # Exposed infection rate compared to the infected (0 the don't infect, 1 the infect in the same rate as the infected )

## Simulation Parameters

In [3]:
# Simulation time
tsim = 1000
# Population
population = 1000000
# Initial Active Infected 
I_act0 = 100

In [4]:
# Mobility
step = 0.05
alpha = list(np.arange(0.5,0.9+step,step))

# Saturation Kinetics Factor
step = 2
kmax = 10
k = [0,5,10,15,20,25,30,35,40]#list(np.arange(0,kmax+step,step))

In [5]:
# Quarantines
qt = 0
qp = 0
iqt = 0
fqt = tsim
sims = [] 

In [6]:
# Parallel Simulation function definition
num_cores = multiprocessing.cpu_count() 
def ParallelSimulation(alpha,k=0, qp = 0, qt = 0,iqt = 0,fqt = 500): 
    simulation = SEIRmodel(beta = beta,mu = mu,ScaleFactor=ScaleFactor,SeroPrevFactor=SeroPrevFactor,expinfection=expinfection,tsim = tsim,tstate='', k = k)
    quarantines = [[tsim, 0.85, alpha, qp, iqt, fqt, qt]] 
    simulation.inputarray = np.array(quarantines) 
    simulation.addquarantine() 
    simulation.initialvalues(I_act0,population,R=0)
    simulation.simulate()  
    return simulation

In [7]:
# Run  Parallel Simulation s
sims = [] 
for i in k: 
    aux = Parallel(n_jobs=num_cores, verbose=50)(delayed(ParallelSimulation)(alpha=j,k=i,qp = qp, qt = qt, iqt = iqt, fqt =fqt) for j in alpha)
    sims.append(aux)



[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:    1.8s
[Parallel(n_jobs=8)]: Done   2 out of   9 | elapsed:    1.9s remaining:    6.5s
[Parallel(n_jobs=8)]: Done   3 out of   9 | elapsed:    1.9s remaining:    3.9s
[Parallel(n_jobs=8)]: Done   4 out of   9 | elapsed:    2.0s remaining:    2.5s
[Parallel(n_jobs=8)]: Done   5 out of   9 | elapsed:    2.3s remaining:    1.8s
[Parallel(n_jobs=8)]: Done   6 out of   9 | elapsed:    2.3s remaining:    1.1s
[Parallel(n_jobs=8)]: Done   7 out of   9 | elapsed:    2.4s remaining:    0.7s
[Parallel(n_jobs=8)]: Done   9 out of   9 | elapsed:    2.4s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   9 out of   9 | elapsed:    2.4s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done   2 out of   9 | elapsed:    0.4s remaining:    1.4s
[Parallel(n_jobs=8

# Analysis
The following contour plot shows how increasing the saturation kinetics factor, reduces the peak size

In [8]:
peak = []
for i in range(len(k)):
    aux = []
    for j in range(len(alpha)):
        aux.append(sims[i][j].peak[0]/sims[0][j].peak[0])
    peak.append(aux)
        

In [9]:
# Contour Plot
fig,ax=plt.subplots(1,1)
cp = ax.contourf(alpha,k,peak) 
fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Peak Size Proportion')
ax.set_xlabel('Mobility')
ax.set_ylabel('Saturation Dynamics Factor')
plt.show() 

## Grid Plot
The following grid plot shows the simulations one by one

In [ ]:
# Grid plot
fig, axs = plt.subplots(len(k), len(alpha))
for i in range(len(k)):
    for j in range(len(alpha)):
        axs[i, j].plot(sims[i][j].t[0],sims[i][j].I[0],label="Infected")  
        axs[i, j].set_title("K: "+str(k[i])+" | Alpha: "+str(alpha[j]))
fig.suptitle('Axes values are scaled individually by default')
#fig.tight_layout()
lines, labels = fig.axes[-1].get_legend_handles_labels()
    
#fig.legend(lines, labels, loc = 'upper center')
fig.legend(lines, labels,loc = 'best')
fig.show()
